Download GNPS data <br>
Replace Job ID below with your GNPS job ID:

In [1]:
! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=b76dd5a123e54a7eb42765499f9163a5&view=download_cytoscape_data' -o GNPS_output_graphML.zip
! unzip -d GNPS_output_graphML/ GNPS_output_graphML.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  437k    0  437k    0     0   202k      0 --:--:--  0:00:02 --:--:--  202k0 --:--:--  0:00:01 --:--:-- 75821
Archive:  GNPS_output_graphML.zip
  inflating: GNPS_output_graphML/METABOLOMICS-SNETS-V2-b76dd5a1-download_cytoscape_data-main.graphml  
  inflating: GNPS_output_graphML/params.xml  
  inflating: GNPS_output_graphML/clusterinfo/99f5516ab61046ec8c8a0c8f035a2880.clusterinfo  
  inflating: GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/5207ac30d6054805bf09e9a49538be08.clustersummary  
  inflating: GNPS_output_graphML/networkedges_selfloop/6f89d6e019364eaba19c2f237fc503db..selfloop  
  inflating: GNPS_output_graphML/result_specnets_DB/562ad714cb0c425c8cd7c00ab4472463.tsv  
  inflating: GNPS_output_graphML/groupmapping_converted/38eb2ddbac514d7384f1ca901558bf8a.group  


Download <i>in silico</i> structural annotation (Dereplicator, Varquest) data <br>
Replace Job IDs below with your Dereplicator or Varquest job IDs:

In [2]:
! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=4d971b8162644e869a68faa35f01b915&view=view_significant' -o Varquest_output.zip
! unzip -d Varquest_output/ Varquest_output.zip

! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=c62d3283752f4f98b1720d0a6d1ee65b&view=view_significant' -o Derep_output.zip
! unzip -d Derep_output/ Derep_output.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30644    0 30644    0     0  26739      0 --:--:--  0:00:01 --:--:-- 26739
Archive:  Varquest_output.zip
  inflating: Varquest_output/DEREPLICATOR-4d971b81-view_significant-main.tsv  
  inflating: Varquest_output/params.xml  
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1348  100  1348    0     0   1466      0 --:--:-- --:--:-- --:--:--  1466
Archive:  Derep_output.zip
  inflating: Derep_output/DEREPLICATOR-c62d3283-view_significant-main.tsv  
  inflating: Derep_output/params.xml  


specify your NAP job ID:

In [1]:
nap_id = 'cb63770fe307410492468f62f9edb8f3'

load libraries

In [2]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = '/Applications/MarvinSuite/bin/' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [3]:
if 'clusterinfo_summary' in os.listdir('GNPS_output_graphML/') and 'DB_result' in os.listdir('GNPS_output_graphML/'):
    netfile = 'GNPS_output_graphML/clusterinfo_summary/' + str(os.listdir('GNPS_output_graphML/clusterinfo_summary/')[0]) 
    gnpslibfile = 'GNPS_output_graphML/DB_result/'+ str(os.listdir('GNPS_output_graphML/DB_result/')[0]) 
elif 'clusterinfosummarygroup_attributes_withIDs_withcomponentID' in os.listdir('GNPS_output_graphML/'):
    netfile = 'GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/' + str(os.listdir('GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[0])
    gnpslibfile = 'GNPS_output_graphML/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML/result_specnets_DB/')[0])
else:
    netfile = 'GNPS_output_graphML/clusterinfosummary/' + str(os.listdir('GNPS_output_graphML/clusterinfosummary/')[0])
    gnpslibfile = 'GNPS_output_graphML/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML/result_specnets_DB/')[0]) 

In [4]:
nap = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t')
derep = pd.read_csv('Derep_output/'+[s for s in os.listdir('Derep_output') if "DEREPLICATOR" in s][0], sep = '\t')
varquest = pd.read_csv('Varquest_output/'+[s for s in os.listdir('Varquest_output') if "DEREPLICATOR" in s][0], sep = '\t')

add all chemical structural information output as dataframe items in list

In [5]:
matches = [gnpslib, nap, derep, varquest]

In [6]:
out = unique_smiles(matches)

In [7]:
out['df'].to_csv("SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [8]:
! molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys.txt

In [9]:
ikeys = pd.read_csv("InchiKeys.txt",  sep='\t',header = None)

out['df']["inchikey"] = ikeys
inchi_dic = make_inchidic(out)

ikeys.columns = ["InChIKey"]
ikeys.to_csv("InchiKeys.txt", quoting=csv.QUOTE_NONE, escapechar='&')

retrieve ClassyFire classifcations

In [10]:
get_classifications("InchiKeys.txt")

1000
2000


In [11]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [12]:
final.head()

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
0,1,S1,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
1,6,S2,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
2,19,S3,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
3,22,S4,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
4,24,S5,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [13]:
final.to_csv("ClassyFireResults_Network.txt", sep = '\t', index = False)

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [14]:
len(set(list(ikeys.InChIKey)) - set(list(df.inchikey)))/len(set(list(ikeys.InChIKey)))

0.015640273704789834

which InChIKeys could not be classified?

In [15]:
set(list(ikeys.InChIKey)) - set(list(df.inchikey))

{'InChIKey=AUUCOMCTAAUNNA-VZUYHUTRSA-N',
 'InChIKey=DBQMOXDLWKVKKG-WBVITSLISA-N',
 'InChIKey=DCUCDCAIOMIBEA-UHFFFAOYSA-O',
 'InChIKey=DHZUMZVZAZYQIW-USYZEHPZSA-N',
 'InChIKey=DIHWZUCEXWUHOD-VZUYHUTRSA-N',
 'InChIKey=DXOJIXGRFSHVKA-UHFFFAOYSA-N',
 'InChIKey=DZPHTQGGRSWHLG-USYZEHPZSA-N',
 'InChIKey=FISVEBQJGBZOPW-USYZEHPZSA-N',
 'InChIKey=FOCMISOLVPZNSV-BRMDALHSSA-N',
 'InChIKey=HJVOAEKCVMNQEX-QZNUWAOFSA-N',
 'InChIKey=IDLUMGMMMVTISV-WBVITSLISA-N',
 'InChIKey=JJISWJXUIHJHGL-RRHRGVEJSA-N',
 'InChIKey=JRDVRIGVIDMJLK-QZNUWAOFSA-N',
 'InChIKey=KQXVERRYBYGQJZ-UHFFFAOYSA-N',
 'InChIKey=LHHCSNFAOIFYRV-UHFFFAOYSA-N',
 'InChIKey=MNOGOEJZCHBAEJ-RWYGWLOXSA-N',
 'InChIKey=NUDVEHBHDBJSMD-UHFFFAOYSA-N',
 'InChIKey=PCKXGFCVDCVIHH-USYZEHPZSA-N',
 'InChIKey=PIRVOZHXFFFFFG-USYZEHPZSA-N',
 'InChIKey=PQEYJXRJLFZSPT-USYZEHPZSA-N',
 'InChIKey=PQIJYJGVCKCIRT-VZUYHUTRSA-N',
 'InChIKey=PTORLBABCFQDMJ-QZNUWAOFSA-N',
 'InChIKey=PWAHARWXEAMNGK-QZNUWAOFSA-N',
 'InChIKey=QBDIXBUIZWZTDA-QZNUWAOFSA-N',
 'InChIKey=SKUXU

create graphML file

In [17]:
if any("FEATURE" in s for s in os.listdir('GNPS_output_graphML/')):
    graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'FEATURE' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
elif any("METABOLOMICS" in s for s in os.listdir('GNPS_output_graphML/')):
    graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'METABOLOMICS' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
else:
    print('There is no graphML file for this GNPS molecular network job')